### Домашняя работа

- Изучите структуру модуля src
- Перенесите функции prefilter_items и postfilter_items из вебинара в модуль src.utils.py
- Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.recommenders.py
- Проверьте, что все модули корректно импортируются


In [1]:
import pandas as pd
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
import implicit
implicit.__version__

'0.5.2'

implicit.__version__ = '0.5.2'

In [3]:
data = pd.read_csv('transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [5]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

In [6]:
q = MainRecommender(data_train, item_features)

C:\anaconda3\lib\site-packages\implicit\utils.py:31: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5000 [00:00<?, ?it/s]

In [7]:
q.get_similar_items_recommendation(150,5)

[1128900, 8011282, 8011285, 822677, 8011285]

In [8]:
q.get_similar_users_recommendation(251,N=4)

[948119, 876548, 1077975, 1033913]

In [9]:
q.get_recommendations(2, N=5)

[854852, 1004906, 1082185, 933835, 1133018]

In [10]:
result['als'] = result['user_id'].apply(lambda x: q.get_recommendations(x, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.13922651933701533